In [ ]:
#Dependencies
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import os
from pprint import pprint

Rafael

Arghavan

In [ ]:
#Cleaning Happiness DF
Happiness =  pd.read_csv('Resources/WorldHappinessReport.csv')
Happiness_New = Happiness.drop(columns=['Delivery Quality', 'Democratic Quality', 
                                        'Standard deviation of ladder by country-year',
                                        'Standard deviation/Mean of ladder by country-year',
                                        'GINI index (World Bank estimate)', 'GINI index (World Bank estimate), average 2000-15',
                                        'gini of household income reported in Gallup, by wp5-year'])

Happiness_Updated = Happiness_New.loc[Happiness_New["year"]==2016,:]
Happiness_Olympics = Happiness_Updated.dropna()
Happiness_Olympics.head()

In [ ]:
#Cleaning Olympics DF
CSV_athletes = pd.read_csv("Resources/120-years-of-olympic-history-athletes-and-results/athlete_events.csv")
CSV_countries =  pd.read_csv("Resources/120-years-of-olympic-history-athletes-and-results/noc_regions.csv")
CSV_athletes = CSV_athletes.drop(columns=['Season', 'City', 'Height', 'Weight'])
CSV_athletes = CSV_athletes.loc[CSV_athletes["Year"]==2016,:]
CSV_athletes_updated = CSV_athletes.fillna("none")
merged_athletes = pd.merge(CSV_athletes_updated, CSV_countries, on = "NOC", how = "left")
merged_athletes = merged_athletes.drop(columns = ['NOC', 'notes'])
merged_athletes1 = merged_athletes.rename(columns= {"region" : "country"})

merged_athletes2 = merged_athletes1.groupby(merged_athletes1["country"])
Final_athletes = merged_athletes2["Name"].count()
Final_df = pd.DataFrame(Final_athletes)
Final_df = Final_df.rename(columns= {"Name" : "Number of Athletes"})
Final_df.head()

In [ ]:
#Creating Final DF
Athlete_Happiness = pd.merge(Final_df, Happiness_Olympics, on = "country")
Athlete_Happiness.to_csv("Outputs/Athlete_Happiness.csv")

Shayan

In [ ]:
happiness_df = pd.read_csv("Resources/WorldHappinessReport.csv")
tourism_df = pd.read_csv("Resources/Tourism.csv", skiprows=3)
happiness17_df = happiness_df.loc[(happiness_df["year"] == 2017)]
happiness_score = happiness17_df[["country", "Life Ladder"]]
happiness_score.columns = ["Country Name", "Happines Score"]
tourism_number = tourism_df[["Country Name", "2017"]]
tourism_number = tourism_number.fillna(0)
merge_table = pd.merge(happiness_score, tourism_number, on = "Country Name")
merge_table.to_csv("Outputs/Tourism_Happiness.csv")


Mariana

In [ ]:
#create a dataframe from a file using pandas
file = 'Resources/Capitals.csv'
# read the CSV in pandas
weather = pd.read_csv(file, encoding="ISO-8859-1")
weather.reset_index(inplace=True)
weather.columns

In [ ]:
# set up additional columns to hold information
weather['Temperature'] = ""

base_url = "http://api.openweathermap.org/data/2.5/weather?"

#print(response['main']['temp'])
# use iterrows to iterate through pandas dataframe
for index, row in weather.iterrows():

    # iterate thru each row of the file
    city = row['Cities']
    url = base_url + "appid=" + "082ff2cde1d396b2b896b2335daad199" + "&q=" + city +"&units=imperial"

# assemble url and make API request
    response = requests.get(url).json()
  
    #update the dataframe 
    try:
        weather.loc[index, 'Temperature'] = response['main']['temp']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#         print(response['main']['temp'])

In [ ]:
weather.head()

Sandra

1. get top songs by country choosing one random day a month for the whole year of 2017

In [ ]:
#create a dataframe from a file using pandas
file = 'Resources/CountryCodesLookup.csv'
#upload country lookup
Countries=pd.read_csv(file,encoding="ISO-8859-1")
#upload dates to look for top songs
spDates= pd.read_csv('Resources/DatesForSpotifyTopSongs.csv')
Countries.columns


In [ ]:
#filter by countries in spotify
spCountries = Countries.loc[Countries["spotifyFlag"] == 1,["COUNTRY","A2 (ISO)"]]

# make a list of countries by dates to get their top 200 songs
#cross join
spCountries["Key"]=1
spDates["Key"]=1
spCountries["Processed"]=0
CountryDate = pd.merge(spCountries, spDates, on="Key").reset_index(drop=True)
#.head(1)
#CountryDate

In [ ]:
#extract top songs by country in 2017. 
# rows get marked as processed. If the script failes, at re-start, it will exclude processed/failed rows.
# this needs to be restarted as many times as it fails.

spCountriesToProcess = CountryDate.loc[(CountryDate["Processed"] == 0) & (CountryDate["COUNTRY"]!='AD' ),:]


from fycharts import SpotifyCharts 
api = SpotifyCharts.SpotifyCharts()


# use iterrows to iterate through pandas dataframe
for index, row in spCountriesToProcess.iterrows():

    # iterate thru each row of the file
    countryCode = str.lower(row['A2 (ISO)'])
    startDate=row["startDate"]
    EndDate=row["EndDate"]
    
    #get data
    print(startDate)  
    print (countryCode)
    CountryDate.loc[index, 'Processed'] = 1
   
       
    api.top200Daily(output_file = 'Outputs/Spotify2017Top.csv', start=startDate, end=EndDate, region=countryCode)
#viral50Daily    
#top200Weekly


2. Query spotify API. Find the songs attributes.  Classify all songs found and saved in the Soptify2017Top.cvs 

In [ ]:
file='Outputs/Spotify2017Top1.csv'
TopSpotify=pd.read_csv(file,encoding="ISO-8859-1")
TopSpotify.columns
TopSpotify["uri"]='spotify:track:'+TopSpotify["id"]
#New columns
TopSpotify["happiness"]=''
TopSpotify["energy"]=''
TopSpotify["danceability"]=''
TopSpotify["loudness"]=''
TopSpotify["speechiness"]=''
TopSpotify["instrumentalness"]=''


TopSpotify.head(1)

In [ ]:

import spotipy

client_id='hiddenCode'
client_secret='HiddenCode'

from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id,client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



# use iterrows to iterate through pandas dataframe
for index, row in TopSpotify.iterrows():

    # iterate thru each row of the file
    tid = row['uri']
    try:
        fearure = sp.audio_features(tid)
        happiness = fearure[0]["valence"]
        energy = fearure[0]["energy"]
        danceability=fearure[0]["danceability"]
        loudness=fearure[0]["loudness"]
        speechiness=fearure[0]["speechiness"]
        instrumentalness=fearure[0]["instrumentalness"]

    #write back to data frame
               
        TopSpotify.loc[index, 'happiness'] = happiness
        TopSpotify.loc[index, 'energy'] = energy
        TopSpotify.loc[index, 'danceability'] = danceability
        TopSpotify.loc[index, 'loudness'] = loudness
        TopSpotify.loc[index, 'speechiness'] = speechiness
        TopSpotify.loc[index, 'instrumentalness'] = instrumentalness
        

    except (KeyError, IndexError, NoneType):
        print(tid)
        print("Missing field/result... skipping.")
TopSpotify.to_csv("Outputs/SpotifyTop2017Final.csv", index=False)